In [2]:
import pandas as pd
import numpy as np
import json
import urllib
import datetime
from time import sleep


In [3]:
def queryBicipublica(biciID, indicador = 'disponibilidad'):
    try:
        url = 'http://epok.buenosaires.gob.ar/getObjectContent/?callback=jQuery19108223016225484279_1505305148763&id=estaciones_de_bicicletas%7C' + biciID
        response = urllib.urlopen(url)
        rawData = response.read().decode("utf-8")
        parsedData = json.loads(rawData.split('(',1)[1][:-1])


        bicisDisponibles = int(parsedData['contenido'][3]['valor'])
        posicionesDisponibles = int(parsedData['contenido'][4]['valor'])
        totales = int(parsedData['contenido'][5]['valor'])

        rotas = 100 - ((bicisDisponibles + posicionesDisponibles)*1. / totales * 100)
        #disponibilidad = bicisDisponibles *1. /  totales * 100
        disponibilidad = bicisDisponibles *1. /  (bicisDisponibles + posicionesDisponibles) * 100
        if disponibilidad > 100.0:
            disponibilidad = 100.0
    except:
        rotas = np.nan
        disponibilidad = np.nan
        
    
    if indicador == 'rotas':
        return rotas
    elif indicador == 'ambas':
        return {'rotas':rotas,'disponibilidad':disponibilidad}
    else:
        return disponibilidad

In [4]:
#base madre
geometria = pd.read_csv('../data/estaciones-de-bicicletas-publicas.csv',sep=';')
#convertir en texto los id de las estaciones para el query
geometria.NRO_EST = ['0' + str(i) if len(str(i)) == 1 else str(i) for i in geometria.NRO_EST]
#remover columnas innecesarias
geometria = geometria.loc[:,['X','Y','NOMBRE','NRO_EST']]
geometria.head()



,X,Y,NOMBRE,NRO_EST
0,-58.392452,-34.583133,FACULTAD DE DERECHO,01
1,-58.374822,-34.592589,RETIRO,02
2,-58.368918,-34.611242,ADUANA,03
3,-58.368950,-34.601721,PLAZA ROMA,04
4,-58.369360,-34.627947,PARQUE LEZAMA,06


In [5]:
datetime.datetime.now().hour

16

In [ ]:
#modulo tiempo
hora = datetime.datetime.now().hour
#poner un dia limite
diaLimite = 18

fechas = []
disponibilidades = []
roturas = []
biciIDs = []

#inicia el loop para cargar data
#cambiar hora 10a 0 y hour por day
while True:
    if datetime.datetime.now().hour == 16:
        print 'hora inicio'
        #mientras el dia actual sea menor al limite
        while datetime.datetime.now().day < diaLimite:
            
            fecha = datetime.datetime.now()
            print 'fecha:',fecha
            #inicia el proceso para cada numero de estacion
            for biciID in geometria.NRO_EST:
                #agrego la id de la bici a la lista
                biciIDs.append(biciID)
                #agrego la disponibilidad de la estacion a la lista
                disponibilidades.append(queryBicipublica(biciID, indicador = 'disponibilidad'))
                #agrego fecha
                fechas.append(fecha)
                
            #espera 15 min
            print fecha
            sleep(60*15)
        break
    else:
        sleep(60)

datos = pd.DataFrame({'NRO_EST':biciIDs,
                      'fecha':fechas,
                      'disponibilidad':disponibilidades#,
                      #'roturas':roturas
                      })

hora inicio
fecha: 2017-09-17 16:59:49.769553
2017-09-17 16:59:49.769553
fecha: 2017-09-17 17:15:44.086710
2017-09-17 17:15:44.086710
fecha: 2017-09-17 17:31:32.390390
2017-09-17 17:31:32.390390
fecha: 2017-09-17 17:48:44.459568
2017-09-17 17:48:44.459568
fecha: 2017-09-17 18:04:43.585951
2017-09-17 18:04:43.585951
fecha: 2017-09-17 18:20:38.914081
2017-09-17 18:20:38.914081
fecha: 2017-09-17 18:36:47.405682
2017-09-17 18:36:47.405682
fecha: 2017-09-17 18:52:47.695310
2017-09-17 18:52:47.695310
fecha: 2017-09-17 19:09:09.397228
2017-09-17 19:09:09.397228
fecha: 2017-09-17 19:25:15.564620
2017-09-17 19:25:15.564620
fecha: 2017-09-17 19:41:48.711471
2017-09-17 19:41:48.711471
fecha: 2017-09-17 19:58:24.201069
2017-09-17 19:58:24.201069
fecha: 2017-09-17 20:14:43.696453
2017-09-17 20:14:43.696453
fecha: 2017-09-17 20:31:04.645944
2017-09-17 20:31:04.645944
fecha: 2017-09-17 20:48:22.129693
2017-09-17 20:48:22.129693
fecha: 2017-09-17 21:04:31.999850
2017-09-17 21:04:31.999850
fecha: 2017-

In [ ]:
print 'datos', datos.shape
print 'geometria',geometria.shape

In [ ]:
data = geometria.merge(datos)
print data.shape
data.head()

In [ ]:
data.to_csv('data.csv')